# PUBG Finish Placement Prediction (Kernels Only)


* Id - 플레이어의 Id
* groupId - 경기 내의 그룹을 식별하는 ID. 현재 그룹의 선수들이 서로 다른 경기에서 경기한다면, 그들은 매번 다른 groupId를 갖게 될 것이다.
* matchId - 경기를 식별하기 위한 ID. train set과 test set에 모두 있는 시합은 없다.
* matchDuration - 경기 시간.
* matchType - 솔로, 듀오와 같은 게임 종류 그 외에는 이벤트게임 -컬럼확인 필요
* assists - 어시스트
* boosts - 부스트 아이템 사용한 수.
 * damageDealt - 가한 데미지 총량. Note: 자신에게 가한 데미지는 제외.
* DBNOs - 빈사상태로 만든 적의 수.
* headshotKills - 헤드샷 으로 처리한 적의 수.
* heals - 치료 아이템 사용 수.
* killPlace - 경기에서 처치한 적의 수 랭킹.
* killPoints - 플레이어의 처치 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, killPoints에서 0은 "없음"으로 처리되어야 한다.
* kills - 처치한 적의 수.
* killStreaks - 단기간에 가장 많이 처치한 적의 최대치.
* longestKill - 플레이어가 적을 죽인 가장 긴 거리. 상대를 죽이고 멀리 운전하는 것이 가장 긴 처치로 이어질 수 있기 때문에 오해의 소지가 있을 수 있다.
* maxPlace - 경기에서 가장 순위가 낮은 것에 대한 데이터. 이것은 순위를 건너뛸 수도 있기 때문에 numGroups와 일치하지 않을 수 있다.
* numGroups - 경기에 있는 팀의 수.
* rankPoints - Elo 방식의 플레이어 랭킹. 다음 버전의 API에서는 삭제될 예정이기 때문에 사용에 주의. '-1' 값은 순위가 "None" 이다.
* revives - 플레이어가 팀원 회복 시킨 수.
* rideDistance - 차량으로 이동한 거리(단위 : 미터).
* roadKills - 차량으로 죽인 플레이어 수.
*  swimDistance - 수영한 거리(단위 : 미터).
* teamKills - 팀킬한 횟수.
* vehicleDestroys - 차량을 폭파시킨 횟수.
* walkDistance - 걸은 총 거리(단위 : 미터).
* weaponsAcquired - 무기 얻은 갯수.
* winPoints - 플레이어의 승리 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, winPoints에서 0은 "없음"으로 처리되어야 한다.
* **winPlacePerc** - 예측 목표. 순위의 퍼센트로 표시되며, 1이면 경기에서 1등이고 0이면 경기에서 꼴지 했다는 것이다. 이것은 numGroups로 계산되는게 아니라, maxPlace로 계산되기 때문에 누락되는 것이 있을수도다.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

dataPath = "./data/"
trainFile = f"{dataPath}train_V2.csv"
testFile = f"{dataPath}test_V2.csv"

# Load DataSet

In [9]:
# _raw : 원본 로드한 데이터
train_raw = pd.read_csv(trainFile)
test_raw = pd.read_csv(testFile)

print(train_raw.shape, test_raw.shape)

(4446966, 29) (1934174, 28)


In [10]:
# _rs : random sample 데이터
train_rs = train_raw.sample(n=10000)
display(train_rs.head())
print(train_rs.shape)

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
4046206,ab867a2b985350,502d97b55b2366,136e2ee8674447,1,1,723.00,4,1,2,3,...,2,714.6,0,0.0,0,0,3276.0,2,1526,0.5769
2295196,836fea0bacc13f,c4e1ba54a979d9,8c5cb116d2f087,0,0,11.23,0,0,0,67,...,0,0.0,0,0.0,0,0,383.1,3,1616,0.3696
2008658,1b9a26fec3c583,39e4878938c641,dbd30a3d2c4aee,0,0,0.00,0,0,0,76,...,0,0.0,0,0.0,0,0,102.2,1,1490,0.2500
357300,50fa91a8a78d31,ada5a4c8072098,18528a92609ef0,1,0,222.50,0,0,0,53,...,0,0.0,0,0.0,0,0,2692.0,7,0,0.5926
1119577,8c828a27a7d1da,0ff0f11f4e8e53,8226cc917a6a25,0,4,380.80,5,0,2,5,...,0,0.0,0,0.0,0,0,2493.0,4,0,0.8400


(10000, 29)


In [11]:
train_rs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 4046206 to 3797868
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               10000 non-null  object 
 1   groupId          10000 non-null  object 
 2   matchId          10000 non-null  object 
 3   assists          10000 non-null  int64  
 4   boosts           10000 non-null  int64  
 5   damageDealt      10000 non-null  float64
 6   DBNOs            10000 non-null  int64  
 7   headshotKills    10000 non-null  int64  
 8   heals            10000 non-null  int64  
 9   killPlace        10000 non-null  int64  
 10  killPoints       10000 non-null  int64  
 11  kills            10000 non-null  int64  
 12  killStreaks      10000 non-null  int64  
 13  longestKill      10000 non-null  float64
 14  matchDuration    10000 non-null  int64  
 15  matchType        10000 non-null  object 
 16  maxPlace         10000 non-null  int64  
 17  numG

In [12]:
train_rs.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.235900,1.139000,131.675248,0.659400,0.22790,1.4073,47.320900,501.928800,0.936300,0.540000,...,0.156200,619.414512,0.003100,4.682812,0.021500,0.008000,1187.015991,3.689900,604.011100,0.480033
std,0.585393,1.738902,175.135383,1.178444,0.61384,2.7630,27.366012,625.580495,1.615891,0.706718,...,0.452573,1497.492019,0.059082,29.193963,0.156334,0.093471,1212.180165,2.418124,739.111606,0.308689
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,24.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,160.275000,2.000000,0.000000,0.204500
50%,0.000000,0.000000,82.500000,0.000000,0.00000,0.0000,47.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,740.700000,3.000000,0.000000,0.475200
75%,0.000000,2.000000,185.925000,1.000000,0.00000,2.0000,71.000000,1170.000000,1.000000,1.000000,...,0.000000,109.900000,0.000000,0.000000,0.000000,0.000000,2008.250000,5.000000,1494.000000,0.750000
max,7.000000,13.000000,2273.000000,15.000000,8.00000,37.0000,100.000000,2063.000000,20.000000,6.000000,...,9.000000,14520.000000,2.000000,641.000000,3.000000,2.000000,9608.000000,42.000000,1960.000000,1.000000


In [13]:
train_rs.describe(include="O")

,Id,groupId,matchId,matchType
count,10000,10000,10000,10000
unique,10000,9975,8987,15
top,ab867a2b985350,1f57ed07adda98,a146204bb96494,squad-fpp
freq,1,2,4,3969


# Data Preprocessing

# Feature engineering